In [6]:
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from joblib import dump, load
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

import joblib
import random

In [7]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
setup_seed(777)
print(torch.cuda.is_available())

True


In [9]:
new_file = 'out_validation.xlsx'

In [10]:
new_dataframe1 = pd.read_excel(new_file, sheet_name=0)

In [11]:
def recall(y_test, y_pred):
    zero_length = len([i for i in y_test if i == 0])

    zero_true_count = 0
    for i in range(len(y_test)):
        if y_test[i] == 0 and y_pred[i] == 0:
            zero_true_count += 1
    return zero_true_count / zero_length

In [12]:
# out_validation
X_second = new_dataframe1.iloc[:,range(0,15)]
y_second = new_dataframe1.iloc[:, -1]
X_second = np.asarray(X_second)
X_second = torch.Tensor(X_second)
y_second = torch.Tensor(y_second)

In [13]:
class Feedforward_second(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward_second, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.batchnorm = torch.nn.BatchNorm1d(self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size, bias=True)
            self.dropout1 = torch.nn.Dropout(p=0.05)
            self.relu1 = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 16, bias=True)
            self.dropout2 = torch.nn.Dropout(p=0.05)
            self.relu2 = torch.nn.ReLU()
            self.fc3 = torch.nn.Linear(16, 1, bias=True)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
            hidden = self.relu(hidden)
            relu = self.relu1(hidden)
            output = self.fc2(relu)
            output = self.relu2(output)
            output = self.fc3(output)
            output = self.sigmoid(output)
            return output

In [16]:
model2 = torch.load("ann_models/model2_outtest2.pt")
Y_OUT = model2(X_second)
Y_OUTs = []

In [17]:
for item in Y_OUT:
    Y_OUTs.append(round(float(item[0])))

In [18]:
print("out validation accuracy:", np.sum(Y_OUTs==np.array(y_second))/len(y_second))

out validation accuracy: 0.8269230769230769


In [19]:
print("out validation recall:", recall(Y_OUTs, y_second))

out validation recall: 0.8888888888888888


In [20]:
fpr2, tpr2, thersholds2 = roc_curve(y_second, Y_OUTs, pos_label=1)
roc_auc2 = auc(fpr2, tpr2)
print("out validation auc:", roc_auc2)

out validation auc: 0.813343328335832
